In [5]:
import pandas as pd
from datatable import dt, f

TODO: Deixar aqui apenas os dados brutos

# Aquisição dos dados brutos

Primeiramente, conforme falado na seção anterior, unimos os microdados de todos os anos para ter um olhar menos fechado em um único ano.

In [9]:
def get_path_table_by_year(year):
  path_table = '/'.join([
    '../data/raw',
    'censo_{year}.zip',
    'Microdados do Censo da Educaç╞o Superior {year}',
    'dados',
    'MICRODADOS_CADASTRO_CURSOS_{year}.CSV'
  ]).format(year = year)
  return path_table

In [10]:
def aggregate_raw_data(from_year, to_year):
  for y in range(from_year, to_year + 1):
    path_table = get_path_table_by_year(y)
    dataset = dt.fread(path_table)
  return dataset

In [11]:
df_raw = aggregate_raw_data(2010, 2019)
display(df_raw)

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_ORGANIZACAO_ACADEMICA,…,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2019,Centro-Oeste,5,Mato Grosso,MT,51,Água Boa,5100201,0,1,…,0,0,0,0,0
1,2019,Centro-Oeste,5,Mato Grosso,MT,51,Água Boa,5100201,0,1,…,0,0,0,0,0
2,2019,Centro-Oeste,5,Mato Grosso,MT,51,Água Boa,5100201,0,1,…,0,0,0,0,0
3,2019,Centro-Oeste,5,Mato Grosso,MT,51,Alto Araguaia,5100300,0,1,…,0,0,0,0,0
4,2019,Centro-Oeste,5,Mato Grosso,MT,51,Aripuanã,5101407,0,1,…,0,0,0,0,0
5,2019,Centro-Oeste,5,Mato Grosso,MT,51,Aripuanã,5101407,0,1,…,0,0,0,0,0
6,2019,Centro-Oeste,5,Mato Grosso,MT,51,Aripuanã,5101407,0,1,…,0,0,0,0,0
7,2019,Centro-Oeste,5,Mato Grosso,MT,51,Barra do Garças,5101803,0,1,…,4,0,0,0,0
8,2019,Centro-Oeste,5,Mato Grosso,MT,51,Barra do Garças,5101803,0,1,…,6,0,0,0,0


Para salvar esses dados agregados em um csv, é necessário fazer o seguinte:

In [19]:
df_raw.to_csv('../data/raw/aggregated_dataset.csv')

Após a agregação, a tabela bruta para trabalho ficou com 246,081 linhas e 196 colunas.  
Após isso, era necessário entender mais sobre as principais características e atributos da base que nos seriam interessantes.